In [ ]:
library(ggplot2)
library(lmtest)
library(urca)
library(sandwich)
library(car)
library(carData)
library(forecast)
library(tseries)
library(dummy)
library(seasonal)
library(hexView)
library(strucchange)

##### Đọc file

File Eview

In [ ]:
readEViews("")

File words

In [ ]:
df = read.table("ch3bt4.txt",header = TRUE)

##### Hồi quy

In [ ]:
hq <- lm(data = df,log(Y)~log(K)+log(L))
summary(hq)

##### Dự báo

In [ ]:
K <- c(50,45)
L <- c(35,32)
predic2 <- predict(hq,data.frame(K,L),interval="confidence",level=0.95)
predic2

##### Kiểm định giả thiết

In [ ]:
linearHypothesis(hq,c("log(K)+log(L)=1","log(L)=0.3"))

Kiểm định phương sai sai số đồng đều

In [ ]:
phandubp <- residual(hq)
summary(lm(data = trang, phandubp ~ AGE + EDUC + I(AGE^2) + I(EDUC^2) + (AGE*EDUC)))

##### Tìm khoảng tin cậy

Cho 1 hệ số

In [ ]:
confint(hoiquy, level = 0.9)

Cho 2 hệ số

$\hat\beta_1+ \hat\beta_2-t_{\alpha/2}se(\hat\beta_1+ \hat\beta_2)<\beta_1+ \beta_2<\hat\beta_1+ \hat\beta_2+t_{\alpha/2}se(\hat\beta_1+ \hat\beta_2)$

In [ ]:
vcov(hq)
var2 <- vcov(hq)[2,2]
var3 <- vcov(hq)[3,3]
cov23 <- vcov(hq)[3,2]

se <- sqrt(var2+var3+2*cov23)
beta2 <- 0.52370 
beta3 <- 0.69301

can_tren <- (beta2+beta3)+qt(0.95,length(df$K)-3)*se
can_duoi <- (beta2+beta3)-qt(0.95,length(df$L)-3)*se

## Biến giả

In [ ]:
anova2 <- lm(data = trang, WAGE ~ BLACK + SOUTH)
summary(anova2)

1. Khi BLACK = 0 và SOUTH = 0 (không phải là người Mĩ gốc Phi và không ở khu vực miền nàm) thì WAGE = 3520.13. Đây chính là mức lương trung bình tháng của nhóm lao động người phi da đen và không ở miền nam.

2. Khi BLACK = 1 và SOUTH = 1 mức lương trung bình của người Mĩ gốc Phi lại còn sống ở miền Nam (tệ phân biệt chủng tộc rất gay gắt ở các ban miền Nam và là một trong những nguyên nhân của nội chiến Mĩ) luôn thấp hơn (221.66+98.89).

3. Khi BLACK = 1 và SOUTH = 0 mức lương trung bình của người Mĩ gốc Phi không sống ở miền Nam được cải thiện hơn chút ít khi mức lương của anh ta chỉ thấp hơn nhóm còn lại 221.66.

4. Khi BLACK = 0 và SOUTH = 1 mức lương trung bình của người Mĩ da trắng (và các màu da khác) thấp hơn 98.89.

5. Tất cả 4 kết luận trên là có ý nghĩa thống kê vì tất cả các hệ số hồi quy đều có ý nghĩa thống kê ở mức cao

#### Sự tương tác của các biến giả

In [ ]:
anova3 <- lm(data = trang, WAGE ~ BLACK + SOUTH + BLACK*SOUTH)
summary(anova3)

Diễn giải mô kết quả của mô hình cũng tương tự như trên với chú ý rằng hệ số hồi quy β4 cho
BLACK*SOUTH là -113.44. Điều này có nghĩa là nếu một quan sát mà anh ta vừa là người da màu, vừa
là dân miền nam thì mức lương của anh ta sẽ thấp hơn so với nhóm còn lại là (155 + 83 + 113)

#### Tương tác giữa biến định lượng và biến định tính

In [ ]:
ancova2 <- lm(data = trang, WAGE ~ URBAN + EDUC + URBAN*EDUC)
summary(ancova2)

Gia tăng thêm một năm học, nếu là lao động ở thành thị thì mức lương sẽ tăng
(38.91 + 26.14) nhưng nếu là lao động ở khu vực khác thì mức lương chỉ tăng 38.91

#### Kiểm định Chow

Cách 1: Tính tay

In [ ]:
nongthon <- lm(WAGE ~ EDUC, data = df, subset = (URBAN == 0))
thanhthi <- lm(WAGE ~ EDUC, data = df, subset = (URBAN == 1))
pooled <- lm(WAGE ~ EDUC, data = df)
RSS <- sum((residuals(pooled))^2)
RSS1 <- sum((residuals(nongthon))^2)
RSS2 <- sum((residuals(thanhthi))^2)
Fqs <- 931*(RSS - RSS1 - RSS2) / (2*(RSS1 + RSS2))
Fqs
## [1] 18.82677

qf(0.95, df1 = 2, df2 = 931)
## [1] 3.005393

Cách 2: TÍnh máy

In [ ]:
sctest(df$ns ~ df$kn, type = "Chow", point = 16)